In [1]:
import random
import networkx as nx
from matplotlib import pyplot as plt
from collections import Counter
import scipy as sc
from scipy.linalg import expm

import math
from utils.plotTools import plot_qwak
import os
import ast
import numpy as np
import json
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import time


from scripts import load_list_from_file, write_list_to_file, load_or_generate_data, draw_graph, draw_graph_from_adjacency_matrix,print_matrix
from scripts_tempHelix import generate_static_temporal_helix, generate_static_temporal_helix2
from scripts_theoreticalHittingTime import create_transition_matrix,expected_hitting_time

In [17]:
def sum_prob_distributions(prob_distributions):
    """
    Sums each element of every probability distribution array in a list and returns a list of these sums.

    Parameters:
    prob_distributions (list of np.array): List of probability distribution arrays.

    Returns:
    list: A list of sums of each probability distribution array.
    """
    sums = [round(np.sum(dist),5) for dist in prob_distributions]
    return sums



In [45]:
def probability_distribution(M_t, init_state):
    """
    Computes the probability distribution p(t) for a graph with Laplacian matrix L,
    starting from the initial state init_state, at time t.
    """
    # Initial probability vector
    p0 = np.zeros(len(M_t))
    p0[init_state] = 1
    
    # Compute the transition matrix M(t)
    # M_t = transition_matrix(L, t)
    
    # Compute the new probability distribution p(t)
    p_t = np.round(np.dot(M_t, p0),5)
    
    return p_t

def efficient_transition_matrix(eigenvalues, eigenvectors, t):
    D_t = np.diag(np.exp(eigenvalues * t))
    return eigenvectors @ D_t @ np.linalg.inv(eigenvectors)

def first_passage_time(G, gamma, start, end, delta_t):
    """
    Simulates a continuous-time random walk on a graph G from start node to end node.
    It uses small time steps delta_t and probability distributions at each step to move to neighboring nodes.
    Returns the time taken to first reach the end node.
    """
    # L = nx.laplacian_matrix(G).toarray() * gamma
    D = np.diag(list(dict(G.degree()).values()))
    A = nx.to_numpy_array(G)
    L = -gamma * (D - A)
    eigenvalues, eigenvectors = sc.linalg.eigh(L)
    p_t_list = [] # vai ser usado para verificar se a dist final de cada passo soma para 1.
    
    current_state = start
    time_elapsed = delta_t
    
    while current_state != end:
        M_t = efficient_transition_matrix(eigenvalues,eigenvectors,time_elapsed)
        p_t = probability_distribution(M_t, current_state)
        p_t_list.append(p_t)
        neighbors = list(G.neighbors(current_state)) + [current_state]
        transition_probabilities = p_t[neighbors]
        if np.sum(transition_probabilities) > 0:
            # Normalizar a prob dist tornando as probs dos neighbors o novo "1".
            transition_probabilities /= np.sum(transition_probabilities)
        else:
            transition_probabilities = np.ones(len(neighbors)) / len(neighbors)
        next_state = np.random.choice(neighbors, p=transition_probabilities)
        current_state = next_state
        time_elapsed += delta_t
        
        for i, row in enumerate(M_t):
            print(f'Row {i+1} sum = {round(np.sum(row),5)}')
        # Sum of individual columns
        for i, col in enumerate(M_t.T):  # P.T is the transpose of P, making columns into rows for easy iteration
            print(f'Column {i+1} sum = {round(np.sum(col),5)}')
    print()
    print()

            
    return time_elapsed,p_t_list


n = 5
G = nx.cycle_graph(n)


# Set the transition rate gamma
gamma = 1.0

# Starting and ending vertices
start_vertex = 0
end_vertex = n//2
delta_t = 0.01

# Perform the walk and get the first-passage time
time_taken,tot_prob_list = first_passage_time(G, gamma, start_vertex, end_vertex, delta_t)
print(sum_prob_distributions(tot_prob_list))

Row 1 sum = 1.0
Row 2 sum = 1.0
Row 3 sum = 1.0
Row 4 sum = 1.0
Row 5 sum = 1.0
Column 1 sum = 1.0
Column 2 sum = 1.0
Column 3 sum = 1.0
Column 4 sum = 1.0
Column 5 sum = 1.0
Row 1 sum = 1.0
Row 2 sum = 1.0
Row 3 sum = 1.0
Row 4 sum = 1.0
Row 5 sum = 1.0
Column 1 sum = 1.0
Column 2 sum = 1.0
Column 3 sum = 1.0
Column 4 sum = 1.0
Column 5 sum = 1.0
Row 1 sum = 1.0
Row 2 sum = 1.0
Row 3 sum = 1.0
Row 4 sum = 1.0
Row 5 sum = 1.0
Column 1 sum = 1.0
Column 2 sum = 1.0
Column 3 sum = 1.0
Column 4 sum = 1.0
Column 5 sum = 1.0
Row 1 sum = 1.0
Row 2 sum = 1.0
Row 3 sum = 1.0
Row 4 sum = 1.0
Row 5 sum = 1.0
Column 1 sum = 1.0
Column 2 sum = 1.0
Column 3 sum = 1.0
Column 4 sum = 1.0
Column 5 sum = 1.0
Row 1 sum = 1.0
Row 2 sum = 1.0
Row 3 sum = 1.0
Row 4 sum = 1.0
Row 5 sum = 1.0
Column 1 sum = 1.0
Column 2 sum = 1.0
Column 3 sum = 1.0
Column 4 sum = 1.0
Column 5 sum = 1.0
Row 1 sum = 1.0
Row 2 sum = 1.0
Row 3 sum = 1.0
Row 4 sum = 1.0
Row 5 sum = 1.0
Column 1 sum = 1.0
Column 2 sum = 1.0
Column 

In [46]:
def efficient_transition_matrix(eigenvalues, eigenvectors, t):
    D_t = np.diag(np.exp(eigenvalues * t))
    return eigenvectors @ D_t @ np.linalg.inv(eigenvectors)


def probability_distribution(M_t, init_state):
    """
    Computes the probability distribution p(t) for a graph with Laplacian matrix L,
    starting from the initial state init_state, at time t.
    """
    # Initial probability vector
    p0 = np.zeros(len(M_t))
    p0[init_state] = 1
    
    # Compute the transition matrix M(t)
    # M_t = transition_matrix(L, t)
    
    # Compute the new probability distribution p(t)
    p_t = np.dot(M_t, p0).round(10)
    
    return p_t

def first_passage_time2(G, gamma, start, end, delta_t):
    """
    Simulates a continuous-time random walk on a graph G from start node to end node.
    It uses small time steps delta_t and probability distributions at each step to move to neighboring nodes.
    Returns the time taken to first reach the end node.
    """
    # Laplacian matrix of the graph
    D = np.diag(list(dict(G.degree()).values()))
    A = nx.to_numpy_array(G)
    L = -gamma * (D - A)
    eigenvalues, eigenvectors = sc.linalg.eigh(L)
    current_state = start
    
    M_t = efficient_transition_matrix(eigenvalues, eigenvectors, 0)
    p_t = probability_distribution(M_t, current_state)
    next_state = np.random.choice(range(len(p_t)), p=p_t)
    p_t_list = []
    
    time_elapsed = delta_t
    while current_state != end:
        M_t = efficient_transition_matrix(eigenvalues, eigenvectors, time_elapsed)
        p_t = probability_distribution(M_t, current_state)
        p_t_list.append(p_t)
        # print(p_t_list)
        # print(p_t)
        next_state = np.random.choice(range(len(p_t)), p=p_t)
        current_state = next_state
        time_elapsed += delta_t
        
        for i, row in enumerate(M_t):
            print(f'Row {i+1} sum = {round(np.sum(row),5)}')
        # Sum of individual columns
        for i, col in enumerate(M_t.T):  # P.T is the transpose of P, making columns into rows for easy iteration
            print(f'Column {i+1} sum = {round(np.sum(col),5)}')
    print()
    print()


    return time_elapsed,p_t_list


n = 5
G = nx.cycle_graph(n)


# Set the transition rate gamma
gamma = 1.0

# Starting and ending vertices
start_vertex = 0
end_vertex = n//2
delta_t = 0.01

# Perform the walk and get the first-passage time
time_taken,tot_prob_list = first_passage_time2(G, gamma, start_vertex, end_vertex, delta_t)
print(sum_prob_distributions(tot_prob_list2))

Row 1 sum = 1.0
Row 2 sum = 1.0
Row 3 sum = 1.0
Row 4 sum = 1.0
Row 5 sum = 1.0
Column 1 sum = 1.0
Column 2 sum = 1.0
Column 3 sum = 1.0
Column 4 sum = 1.0
Column 5 sum = 1.0
Row 1 sum = 1.0
Row 2 sum = 1.0
Row 3 sum = 1.0
Row 4 sum = 1.0
Row 5 sum = 1.0
Column 1 sum = 1.0
Column 2 sum = 1.0
Column 3 sum = 1.0
Column 4 sum = 1.0
Column 5 sum = 1.0
Row 1 sum = 1.0
Row 2 sum = 1.0
Row 3 sum = 1.0
Row 4 sum = 1.0
Row 5 sum = 1.0
Column 1 sum = 1.0
Column 2 sum = 1.0
Column 3 sum = 1.0
Column 4 sum = 1.0
Column 5 sum = 1.0
Row 1 sum = 1.0
Row 2 sum = 1.0
Row 3 sum = 1.0
Row 4 sum = 1.0
Row 5 sum = 1.0
Column 1 sum = 1.0
Column 2 sum = 1.0
Column 3 sum = 1.0
Column 4 sum = 1.0
Column 5 sum = 1.0
Row 1 sum = 1.0
Row 2 sum = 1.0
Row 3 sum = 1.0
Row 4 sum = 1.0
Row 5 sum = 1.0
Column 1 sum = 1.0
Column 2 sum = 1.0
Column 3 sum = 1.0
Column 4 sum = 1.0
Column 5 sum = 1.0
Row 1 sum = 1.0
Row 2 sum = 1.0
Row 3 sum = 1.0
Row 4 sum = 1.0
Row 5 sum = 1.0
Column 1 sum = 1.0
Column 2 sum = 1.0
Column 

In [47]:
def theoretical_hitting_time(P, init, target):
    """
    Calculate the expected hitting time to state z.

    Parameters:
    P (numpy.ndarray): Transition matrix.
    q (numpy.ndarray): Initial state distribution.
    z (int): Target state.

    Returns:
    float: Expected hitting time to state z from state q.
    """
    
    n = P.shape[0]  # Number of states
    # print(n)
    
    if target=='mid':
        target = n//2
        
    if target=='-1':
        target = n-1
        
    order = list(range(n))
    order.remove(target)
    order.append(target)
    
    P = P[np.ix_(order, order)]
    # print_matrix(P)
    # print()
        
    P_prime = P[:-1, :-1]
    
    print('P Prime')
    print_matrix(P_prime)

    init_dist = np.zeros(n-1)
    init_dist[init] = 1
    
    target_dist = np.ones(n-1)

    I = np.eye(n-1)

    P_prime_inv = np.linalg.inv(I - P_prime)
    
    print('P Prime inv')
    print_matrix(P_prime_inv)
    print()
        
    hitting_time = np.dot(init_dist, np.dot(P_prime_inv, target_dist))

    return hitting_time

def continuous_theoretical_hitting_times_for_cycle_graphs( graphList, gamma, delta_t, init):
    hitting_times = []
    for G in graphList:
            D = np.diag(list(dict(G.degree()).values()))
            A = nx.to_numpy_array(G)
            L = -gamma * (D - A)
            # print(f'm = {len(L)} \t trace = {np.trace(-L)})')
            P = expm(L * delta_t)
            print('P')
            print_matrix(P)
            # Sum of individual rows
            for i, row in enumerate(P):
                print(f'Row {i+1} sum = {np.sum(row)}')

            # Sum of individual columns
            for i, col in enumerate(P.T):  # P.T is the transpose of P, making columns into rows for easy iteration
                print(f'Column {i+1} sum = {np.sum(col)}')
                
            target = 'mid'
            hitting_time = theoretical_hitting_time(P, init, target)
            hitting_times.append(hitting_time)
    return hitting_times

n_range = range(4, 5)
gamma = 1.0
delta_t = 0.01
init = 0
graphList = [nx.cycle_graph(n) for n in n_range]
theor_time = continuous_theoretical_hitting_times_for_cycle_graphs(graphList, gamma, delta_t, init)


P
|0.98 0.01 0.00 0.01 |
|0.01 0.98 0.01 0.00 |
|0.00 0.01 0.98 0.01 |
|0.01 0.00 0.01 0.98 |
Row 1 sum = 1.0
Row 2 sum = 1.0
Row 3 sum = 1.0
Row 4 sum = 1.0
Column 1 sum = 1.0
Column 2 sum = 1.0
Column 3 sum = 1.0
Column 4 sum = 1.0
P Prime
|0.98 0.01 0.01 |
|0.01 0.98 0.00 |
|0.01 0.00 0.98 |
P Prime inv
|101.00 50.50 50.50 |
|50.50 76.00 25.50 |
|50.50 25.50 76.00 |

